# UI to conduct a manual audit of the labels from conventions classifiers. 


# ***IF NOT USING BINDER - UNCHECK THE BOX***


The output of the classifier via Iterative_Models_building.ipyng are stored to

    -file 'audit_training_data.gz (which is a zipped pandas database) 
    -location: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/

The UI randomly displays an already classified sentence for the purpose of audit and correction

Completed audits will be stored locally until the "Save to GIT" button is selected, in which case the tsv of audit will be stored to 

    - file 'audit_training_data-{gituser}{date}.tsv 
    -location: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/

Content of the Audited files may be coccat and added to the Train_Validate data.


In [1]:
import os
import re
import random
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display, clear_output, HTML, Image, IFrame
import base64
import io
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import base64
from github import Github, GithubException
from github import InputGitTreeElement
import requests
import time
from io import BytesIO
from PIL import Image
import urllib.request


In [2]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
FLAG__ON_BINDER = True
FLAG__FROM_GIT = True
DOWNLOAD_TSV_PATH = './Data/Iterative-models-building/Classification results/Conventions/Audited/'


########These are all defined in the shared helper files should modify to point there####
RESOURCES_DIR_LOCAL = 'Data/Iterative-models-building/Resources/'
RESOURCES_DIR_GIT = 'cp_wssc/Data/Iterative-models-building/Resources/'
RESOURCES_URL_GIT = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Resources/'
CLASSIFIED_DATA_DIR_LOCAL  = './Data/Iterative-models-building/Classification results/Conventions/'
CLASSIFIED_DATA_DIR_GIT =  'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
AUDITED_DATA_DIR_LOCAL  = './Data/Iterative-models-building/Classification results/Conventions/Audited'
AUDITED_DATA_DIR_GIT =  'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited'
###########                 

my_style = {'description_width': 'initial'}
my_layout = {'width': '600px'}
my_layout_short = {'width': '300px'}
my_layout_shortest = {'width': '200px'}

In [5]:

box_binder = widgets.Checkbox(True, description='*MUST un-check thid box IF NOT running on binder', style = my_style, layout = my_layout)

def update_box_binder(change):
    global FLAG__ON_BINDER
    global FILE_PATH
    global DOWNLOAD_PATH
    FILE_NAME = 'audit_training_data.gz'
    FLAG__ON_BINDER = change['new']
    if (FLAG__ON_BINDER == True):
        DOWNLOAD_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
        DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
        FILE_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'

    else:
        DOWNLOAD_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
        DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
        FILE_PATH = './Data/Iterative-models-building/Classification results/Conventions/Audited/'
        if not os.path.exists(DOWNLOAD_PATH):
            os.makedirs(DOWNLOAD_PATH)

        if not os.path.exists(FILE_PATH):
            os.makedirs(FILE_PATH)
    print (FILE_PATH)
   
    
box_binder.observe(update_box_binder, 'value')

display(box_binder)
print("....loading test tool")

Checkbox(value=True, description='*MUST un-check thid box IF NOT running on binder', layout=Layout(width='600p…

....loading test tool
./Data/Iterative-models-building/Classification results/Conventions/Audited/


In [6]:
FILE_NAME = 'audit_training_data.gz'
TOTAL_RECORDS_AUDITED = 0
TOTAL_UNSAVED_AUDITS = 0
LAST_SAVED = ""
LAST_FILE_SAVED = ""

if FLAG__ON_BINDER == True:
    DOWNLOAD_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
    DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
    FILE_PATH = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'
    
else:
    DOWNLOAD_PATH = './Data/Iterative-models-building/Classification results/Conventions/'
    DOWNLOAD_PATH = DOWNLOAD_PATH + FILE_NAME
    FILE_PATH = './Data/Iterative-models-building/Classification results/Conventions/Audited/'
    
if FLAG__FROM_GIT == True:
    DOWNLOAD_PATH = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/audit_training_data.gz?raw=true'
    GIT_PUSH_PATH = 'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/' 
    IMAGE_PATH = 'https://github.com/aideenf/AIVC/raw/master/cp_wssc/Data/Iterative-models-building/Resources/labelled_data_proportions.png'
    

In [7]:
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '15px'),
  ('text-align', 'center'),
  #('font-weight', 'bold'),
  ('color', 'grey'),
  ('background-color', 'lightgrey'),
  ('border-color', 'black'),
  ('border-style' ,'solid') ,
  ('border-width', '1px'),
  ('border-collapse','collapse'),
  ('max-width', '700px'),
  ('min-width', '700px')
  ]
  

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '15px'),
  ('text-align', 'left'),
  #('font-weight', 'bold'),
  ('color', 'lightgrey'),
  ('background-color', 'grey'),
  ('border-color', 'black'),
  ('border-style' ,'solid') ,
  ('border-width', '1px'),
  ('border-collapse','collapse'),
  ('height', '100px'),
  ('max-width', '700px'),
  ('min-width', '700px')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]


In [8]:
def image_to_byte_array(image:Image):
  imgByteArr = io.BytesIO()
  image.save(imgByteArr, format=image.format)
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr


def retrieve_classified_data (path):
   
    if FLAG__FROM_GIT == True:
        # sep='\t'  to specify tab seperated?
        df = pd.read_csv(path, error_bad_lines=False, compression='gzip', header=0, quotechar='"')

    if FLAG__FROM_GIT == False:
        df = pd.read_csv(path, sep='\t')

    df = df.drop(df.columns[0], axis=1)
    
    #print("num of records with NaN", df.isna().sum() )
    
    #Drop any rows where there is 0 confidence value. 
    df = df.dropna(subset=['confidence_value'])
    #if source is not specified, fill as other. 
    df['data_provenance'] = df.data_provenance.fillna('other')
    row_to_edit = df.sample(n = 1)
    index_number = 0     
    #print("num of records with NaN", df.isna().sum() )   
    #remove rows where text = "unknown"
    df = df[df['text']!="unknown"]
    
    print("data file loaded:", len(df.index), " rows")
    return df



    # Method to get the latest aggregate audited data and to return a pandas DB containing the aggregated content.

def get_aggregated_classified_data(self, use_git,
                                    data_dir_git,
                                    data_dir_local,
                                    github__url,
                                    git_owner,
                                    git_repo):
    use_git = use_git
    data_dir_git = data_dir_git
    data_dir_local = data_dir_local
    github_url_path = github_url
    git_owner = git_owner
    git_repo = git_repo

    aivm_helper = AIVM_helper()

    files = []
    data = {}
    all_df = pd.DataFrame(columns=['text', 'provenance'])

    if (use_git == True):
        files = aivm_helper.list_files_from_github_dir(git_owner, git_repo, data_dir_git)
        files = aivm_helper.remove_excluded_files(files)
        for f in files:
            data[aivm_helper.clean_file_name(f)] = pd.read_csv(github_url_path+f, sep='\t', error_bad_lines=False)
            # NOTE: "truncated": false  we should check for truncated = true to do follow on call to get all files
            tmp_df = data[aivm_helper.clean_file_name(f)]
            tmp_df = tmp_df.rename(columns={"sentence": "text"})
            all_df = all_df.append(tmp_df)

    elif (use_git == False):
        # For each file() in gathered conventions folder
        files = [f for f in os.listdir(data_dir_local)]
        files = aivm_helper.remove_excluded_files(files)
        for f in files:
            data[aivm_helper.clean_file_name(f)] = pd.read_csv(os.path.join(data_dir_local, f), sep='\t')
            tmp_df = data[aivm_helper.clean_file_name(f)]
            tmp_df = tmp_df.rename(columns={"sentence": "text"})
            all_df = all_df.append(tmp_df)

    gc.collect()
    return files, all_df


# Read the key words from GIT

In [9]:
urlBase  = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Resources/keywords_df.csv'
dfKeyWords = pd.read_csv(urlBase, index_col = False)
list_conv = list (pd.unique(dfKeyWords['convention']))
conventions = list (pd.unique(dfKeyWords['convention']))
list_conv.sort()
conventions.sort()
keyword_de_dict = {}
keyword_td_dict = {}
for conv in list_conv:
    keyword_de_dict[conv] = dfKeyWords.loc[(dfKeyWords['convention'] == conv) & (dfKeyWords['source'] == "domain_expert"), 'word'].tolist()
    keyword_td_dict[conv] = dfKeyWords.loc[ (dfKeyWords['convention'] == conv) & (dfKeyWords['source'] != "domain_expert"), 'word'].tolist()

In [15]:
list_conv = np.insert(list_conv, 0, "text")
list_conv = np.insert(list_conv, 1, "provenance")

audited_df = pd.DataFrame(columns=list_conv)

url_gitHub = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/git_ai_repo_for_audit_tool.tsv?raw=true'
url_reddit = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/reddit_ai_for_audit_tool.tsv?raw=true'

convention_sentences_df = pd.read_csv(url_gitHub, sep = '\t', index_col = False)
convention_sentences_reddit_df = pd.read_csv(url_reddit, sep = '\t', index_col = False)

# classified_data_files, classified_data_df = model_helpers.get_aggregated_gathered_data (
#                             USE_GIT_HUB,
#                             paths.GATHERED_DATA_CONV_DIR_GIT,
#                             paths.GATHERED_DATA_CONV_DIR_LOCAL,
#                             paths.GITHUB_GATHERED_URL_PATH, 
#                             paths.GIT_OWNER,
#                             paths.GIT_REPO)


# display (convention_sentences_df.head(1)) 
# display (convention_sentences_reddit_df.head(1)) 
# display (convention_sentences_df.shape[0]) 
# display (convention_sentences_reddit_df.shape[0]) 

convention_sentences_df = pd.concat([convention_sentences_df,convention_sentences_reddit_df], ignore_index=True)
# display (convention_sentences_df.head(1)) 
# display (convention_sentences_df.shape[0])

,Unnamed: 0,text,ref,repo,data_provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,0,Tensorflow is an end-to-end open source platfo...,45717250,tensorflow/tensorflow,github_ai,0.0,0.121212,0.0,0.4,0.0,0.127273,0.0,0.019608


,Unnamed: 0,text,ref,repo,data_provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,0,Original video + Stennifer Lawrscemi.,alrasv,https://www.reddit.com/r/artificial/comments/a...,reddit_ai,0.085366,0.108108,0.092593,0.25,0.025,0.041667,0.117647,0.081967


131964

38135

,Unnamed: 0,text,ref,repo,data_provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,0,Tensorflow is an end-to-end open source platfo...,45717250,tensorflow/tensorflow,github_ai,0.0,0.121212,0.0,0.4,0.0,0.127273,0.0,0.019608


170099

In [16]:
def time_stamp():
    now = datetime.now()
    timestamp = datetime.timestamp(now)
    dt_object = datetime.fromtimestamp(timestamp)
    words = str(dt_object).split(' ');
    return words[0], words[1]

def save_to_github(git_user, git_password, git_owner_repo, my_file_list, push_to_git_as):
    '''
    in order to push a file to github it must first be stored locally, then pushed
    this local location can also be local to a virtual machine. 
    takes: 
            git username, password, repo, 
            a list of files to push to git ie the full local location of file,
            a matching list of paths to push each file to in Git hub 
    '''
    user = git_user
    password = git_password
    url = git_owner_repo #owner/repo eg aideenf/AIVC
    file_list = []  #push these list of files to git
    file_names = [] #push to this location in git
    message = 'ok'
    
    
    try:
        g = Github(user, password)
        try:
            repo = g.get_repo(url)
        except (IOError, OSError, GithubException) as e: 
            return "error", e.message

        file_list = my_file_list 
        file_names = push_to_git_as
                 
        commit_message = 'training data updated via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):   
            with open(entry) as input_file:
                #data = input_file.read()   #works with non zip file
                data = base64.b64encode(open(entry, "rb").read())
                
            if entry.endswith('.png'):
                data = base64.b64encode(data) 
                

            blob = repo.create_git_blob(data.decode("utf-8"), "base64")
            element = InputGitTreeElement(path=file_names[i], mode='100644', type='blob', sha=blob.sha)

            #push to git as file_names[i]
            #print ("push to git as:", file_names[i])           
            #element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
        
        
            #element_list is a list of InputGitTreeElement. 
            #Each one corresponds to a file. 
            # the 'content' of InputGitTreeElement can only be of type 'str' or 'unicode'. 
            #When I load a file to memory I have type 'bytes'. 
            #What is the right way to encode those bytes to str or unicode to upload a .zip
            element_list.append(element)   
        
        tree = repo.create_git_tree(element_list, base_tree)
        parent = repo.get_git_commit(master_sha)
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        return commit, message 
    except:
        message = "GitHub save FAILED:" + '\n' +"Are your github login credentials correct?" + '\n' + "Are you a collaberator in the repo?"
        return "error", message


In [17]:

checkbox_dict = {}
cb_list = []
for conv in conventions:
    checkbox_dict[conv] = widgets.Checkbox(value=False,description=conv,disabled=False, style = my_style,
    layout = my_layout_shortest)
    cb_list.append(checkbox_dict[conv])
    
check_boxes  = widgets.HBox(cb_list)


###### Confidence values
min_confidence = .5 #below this threshold the classification will be negative, above it will be pos
max_confidence = 1


selectHTMLHead = widgets.HTML(
            "<font color = '#8c8c8c'><h3><center>*** Model Classification, Audit Tool ***</center></h3></font>")

selectHTML = widgets.HTML(
            "<font color = '#8c8c8c'><h4><left>--- Filter ---</left></h4></font>")

saveHTML = widgets.HTML(
            "<h4><left><font color = '#8c8c8c'>--- Push unsaved audits to Github repo ---</font></left></h4>")

actionHTML = widgets.HTML(
            "<font color = '#8c8c8c'><h4><left>--- Action ---</h4></left></font>")


space = widgets.Label('  ', layout=widgets.Layout(width='100%'))
small_space = widgets.Label(' ', layout=widgets.Layout(width='5%'))



convention_drop_down = widgets.Dropdown(
    options={ 
             'Civic':'convention_civic', 
             'Domestic':'convention_domestic', 
             'Green':'convention_green', 
             'Industrial':'convention_industrial', 
             'Inspired':'convention_inspired',
             'Market':'convention_market', 
             'Project':'convention_project',
             'Renown':'convention_renown',
             'None':'None'},
    value='convention_civic',
    description='Convention  :',
    disabled=False,
    style = my_style,
    layout = my_layout
)

source_drop_down = widgets.Dropdown(
    options =  convention_sentences_df.data_provenance.unique(),
    description="Data source :",
     #value=None,
    disabled=False,
    style = my_style,
    layout = my_layout
)



convention_pos_neg = widgets.Dropdown(
    options=['Positive', 'Negative'],
    value='Positive',
    description='Classified as:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

confidence_score_slider_min = widgets.FloatSlider(
    min=min_confidence, 
    max=max_confidence,
    value = min_confidence,
    step = 0.1,
    description='Min Confidence level:',
    style = my_style,
    layout = my_layout
    )
confidence_score_slider_min.style.handle_color = '#5c85d6'


confidence_score_slider_max = widgets.FloatSlider(
    min=min_confidence, 
    max=max_confidence, 
    value = max_confidence,
    step = 0.1,
    description='Max Confidence level:',
    style = my_style,
    layout = my_layout
    )
confidence_score_slider_max.style.handle_color = '#5c85d6'



change_value_button = widgets.Button(
            description= "Save",
            disabled=False,
            button_style='',
            tooltip='Audit complete, change classification',
        )
change_value_button.style.button_color = '#e68a00'



# keep_value_button = widgets.Button(
#             description="Save" ,
#             disabled=False,
#             button_style='',
#             tooltip='Audit complete, already classified ok',

#         )
# keep_value_button.style.button_color = '#00b386'


audit_next_button = widgets.Button(
            description='Skip',
            disabled=False,
            button_style='',
            tooltip='Move to next, without audit'
        )
audit_next_button.style.button_color = '#d9d9d9'


undo_last_audit = widgets.Button(
            description='Undo last',
            disabled=False,
            button_style='',
            tooltip='undoes the last line added to audit'
        )
audit_next_button.style.button_color = '#d9d9d9'


save_to_git_button = widgets.Button(
            description='Save to GitHub',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )
save_to_git_button.style.button_color = '#00b386'



download_button = widgets.Button(
            description='Download',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )
download_button.style.button_color = '#d9d9d9'



user = widgets.Text(
            description='Username',
            disabled=False,
            layout=my_layout_shortest,
            tooltip='Enter your GitHub username',
            style = my_style
        )


pswd = widgets.Password(
            description='Password',
            disabled=False,
            layout=my_layout_shortest,
            tooltip='Enter your GitHub password',
            style = my_style
        )


def change_value_button_clicked(b):
    
    global TOTAL_UNSAVED_AUDITS 
    global TOTAL_RECORDS_AUDITED 
    global index_number
    global row_to_edit
    global audited_df
    global convention_sentences_df
    
        
    text = row_to_edit.text.values.astype(str)[0]
    prov = row_to_edit.data_provenance.values.astype(str)[0]
    current_conv = convention_drop_down.value.replace('convention_', '')
    
    
    current = pd.DataFrame({
                            'text': [text],
                            'provenance': [prov],
                            'civic': [ checkbox_dict['civic'].value ],
                            'domestic': [checkbox_dict['domestic'].value],
                            'green': [checkbox_dict['green'].value],
                            'industrial': [checkbox_dict['industrial'].value],
                            'inspired': [checkbox_dict['inspired'].value],
                            'market': [checkbox_dict['market'].value],
                            'project': [checkbox_dict['project'].value],
                            'renown': [checkbox_dict['renown'].value]
                            })
   
    for con in conventions:
        current[con] = current[con].astype(int)
    
    audited_df = audited_df.append(current, ignore_index=True)



    
    TOTAL_UNSAVED_AUDITS = TOTAL_UNSAVED_AUDITS + 1
    TOTAL_RECORDS_AUDITED = TOTAL_RECORDS_AUDITED + 1
    with output:
        clear_output(wait=True)
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value, convention_pos_neg.value )
    
    with outputToSave:
        clear_output(wait=True)
        display (audited_df)
        
    with outputGitHub:
        clear_output(wait=True)
        print ("")
        

def audit_next_button_clicked(b):
    with output:
        clear_output(wait=True)
        output_function(convention_drop_down.value, confidence_score_slider_min.value, confidence_score_slider_max.value, source_drop_down.value, convention_pos_neg.value )
    with outputToSave:
        clear_output(wait=True)
        display (audited_df)
    with outputGitHub:
        clear_output(wait=True)
        print("")
        
def undo_last_audit_button_clicked(b):
    global audited_df
    global TOTAL_UNSAVED_AUDITS 
    global TOTAL_RECORDS_AUDITED

    if (audited_df.shape[0] > 0):
        audited_df.drop(audited_df.tail(1).index,inplace=True) 
        with outputToSave:
            clear_output(wait=True)
            display (audited_df)   
        TOTAL_UNSAVED_AUDITS = TOTAL_UNSAVED_AUDITS - 1
        with outputCount:
            clear_output(wait=True)
            print("Total audited:", TOTAL_RECORDS_AUDITED, " Total unsaved:", TOTAL_UNSAVED_AUDITS," Last Saved:", LAST_SAVED )
       
    else:
        with outputToSave:
            clear_output(wait=True)
            print ("No more rows to delete")
            
            
def download_button_clicked(b):
    global FILE_PATH
    global LAST_FILE_SAVED
    file_name = DOWNLOAD_TSV_PATH + LAST_FILE_SAVED
    display(IFrame(file_name, width=700, height=400))

def save_changes_to_github_button_clicked(b):
    global TOTAL_UNSAVED_AUDITS 
    global convention_sentences_df
    global LAST_SAVED
    global LAST_FILE_SAVED
    global FILE_PATH
    global GIT_PUSH_PATH
    global audited_df
    GIT_USER = user.value
    GIT_PSWD = pswd.value
    GIT_REPO = 'aideenf/AIVC'
    D, T = time_stamp()
    file_name = 'audited_training_data_'+ GIT_USER   + '_' + D + '-' + T +  '.tsv'
    file_name_zip = 'audit_training_data_'+ GIT_USER + '_' + D + '-' + T +   '.gz'
    LAST_FILE_SAVED = file_name
    with outputGitHub:
            clear_output(wait=True)
            display("saving, please wait....")
    try:
        audited_df.to_csv(FILE_PATH + file_name, sep = '\t', index = False)
    except (IOError, OSError) as e:
        with outputGitHub:
            clear_output(wait=True)
            display(HTML("<font color='red'><b>Warning!!</b></font>"))
            print ("Are your settings correct for local machine v's binder?")
            print (e)
        return "error"
        
    my_file_list = [FILE_PATH + file_name]
    push_to_git_as = [GIT_PUSH_PATH + file_name]
    commit, message = save_to_github(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)
    if (commit != "error"):
        with outputGitHub:
            clear_output(wait=True)
            print ("File to commit:", my_file_list[0])
            print ("Push to git as:", push_to_git_as[0])
            print ("Commit: ", commit)
            TOTAL_UNSAVED_AUDITS = 0
            D, T = time_stamp()
            LAST_SAVED = D+T
            print ("Number of audits saved: ", audited_df.shape[0] )
            display(HTML("<font color='green'><b>Saved!!</b></font>"))
            audited_df = pd.DataFrame(columns=['text', 'provenance', 'convention', 'old', 'new'])
    if (commit == "error"):
        with outputGitHub:
            clear_output(wait=True)
            print ("File to commit: ", my_file_list[0])
            print ("Push to git as: ", push_to_git_as[0])
            display(HTML("<font color='red'><b>Warning!!</b></font>"))
            print(message)
            print ("")
            print("If you continue to have a problem download here and save locally")
            display(download_button)
            
            

def output_function(convention_drop_down_value, confidence_score_slider_min_value, confidence_score_slider_max_value, source_drop_down_value, convention_pos_neg_value ):
    global convention_sentences_df
    global row_to_edit
    global index_number
    global LAST_SAVED
    global TOTAL_RECORDS_AUDITED
    global TOTAL_UNSAVED_AUDIT
    
    
    #Confidence values in the case that .05 is the threshold between Negative and Positive
    confidenceOfPosMin = confidence_score_slider_min.value #0.5
    confidenceOfPosMax = confidence_score_slider_max.value  #somewhere around 1
    confidenceOfNegMin = 0
    confidenceOfNegMax = .5
    
    filter_lower = 0
    filter_upper = 1
    
    if (convention_pos_neg.value ==  'Positive'):

        filter_lower = confidenceOfPosMin
        filter_upper = confidenceOfPosMax

    if (convention_pos_neg.value ==  'Negative'):

        filter_lower = confidenceOfNegMin
        filter_upper = confidenceOfNegMax
    
   
    convention = convention_drop_down_value.replace('convention_', '')



    
    try:
        
        if convention_drop_down_value == "None":
            row_to_edit = convention_sentences_df.loc[
                                            (convention_sentences_df['data_provenance'] == source_drop_down_value) &
                                            (convention_sentences_df['convention_civic'] < 0.5) &
                                            (convention_sentences_df['convention_domestic'] < 0.5) &
                                            (convention_sentences_df['convention_green'] < 0.5) &
                                            (convention_sentences_df['convention_industrial'] < 0.5)&
                                            (convention_sentences_df['convention_inspired'] < 0.5) &
                                            (convention_sentences_df['convention_market'] < 0.5) &
                                            (convention_sentences_df['convention_project'] < 0.5) &
                                            (convention_sentences_df['convention_renown'] < 0.5)].sample(n = 1)
            num_of_examples = convention_sentences_df.loc[
                                            (convention_sentences_df['data_provenance'] == source_drop_down_value) &
                                            (convention_sentences_df['convention_civic'] < 0.5) &
                                            (convention_sentences_df['convention_domestic'] < 0.5) &
                                            (convention_sentences_df['convention_green'] < 0.5) &
                                            (convention_sentences_df['convention_industrial'] < 0.5) &
                                            (convention_sentences_df['convention_inspired'] < 0.5) &
                                            (convention_sentences_df['convention_market'] < 0.5) &
                                            (convention_sentences_df['convention_project'] < 0.5) &
                                            (convention_sentences_df['convention_renown'] < 0.5)].shape[0]
            
            
        else:    
            row_to_edit = convention_sentences_df.loc[
                                                (convention_sentences_df['data_provenance'] == source_drop_down_value) &
                                                (convention_sentences_df[convention_drop_down_value] >= filter_lower) &
                                                (convention_sentences_df[convention_drop_down_value] <= filter_upper)].sample(n = 1)
        
            num_of_examples = convention_sentences_df.loc[
                                                (convention_sentences_df['data_provenance'] == source_drop_down_value) &
                                                (convention_sentences_df[convention_drop_down_value] >= filter_lower) &
                                                (convention_sentences_df[convention_drop_down_value] <= filter_upper)].shape[0]

        txt = "The number of filtered samples is " + str(num_of_examples)
        display(HTML("<font color='green'>" + txt + "</font>"))
        if (num_of_examples < 20):
            display(HTML("<font color='green'><b>Warning!</b> " + "The sample number is low and you may see repeats" + "</font>"))
    except ValueError:
        print (ValueError)
        display(HTML("<font color='red'><b>Warning</b> " + "No data, please adjust filter:</font>"))
    
    if not row_to_edit.empty:
        display (row_to_edit[['text']].style.background_gradient().set_caption('Row randomly selected based on filter').set_table_styles(styles).hide_index())
        sentence = row_to_edit['text'].values.astype(str)[0]
        try:
            conf = row_to_edit[convention_drop_down_value].values.astype(float)[0]
        except:
            conf = 0
        res = ""
        ref = row_to_edit['ref'].values.astype(str)[0]
        repo = row_to_edit['repo'].values.astype(str)[0]
        source = row_to_edit['data_provenance'].values.astype(str)[0]
        confScore = 0
        if (conf > .5):
            res = 'POSITIVE'
            confScore = conf

        if (conf < .5):
            res = 'NEGATIVE'
            confScore = 1 - conf
        
        
        if source == "github_ai":
                repo1 = 'https://github.com/' + repo + '/blob/master/README.md'
                repo2 = 'https://github.com/' + repo + '/blob/master/README.rst'

        true_for = ""
        for conv in conventions:
            to_dis = str(round(row_to_edit['convention_'+conv].values[0].item(),2))
            checkbox_dict[conv].description = conv + " (" + to_dis + " )"
            val = round ( row_to_edit['convention_'+conv].values[0]).item()
            checkbox_dict[conv].value = bool(val)
            if bool(val) == True:
                true_for = true_for + conv + ", "
        #display(HTML("Sentence: <b><font color='green'>" + sentence + "</b></font>" ))
        display(HTML("Text field classified as: <b><font color='black'>" + res + "</b> " + "</font> for the <font color='black'><b>" +  true_for.upper() + " </b></font>economy of conventions </> "))
        display(HTML("Reference :" + ref))
        
        display(HTML("Link: " + repo1))
        if source == "github_ai":
            display(HTML("Link :" + repo2))
            
        
        index_number = row_to_edit.index.values.astype(int)[0]
        try:
            with outputCount:
                clear_output(wait=True)
                print("Total audited:", TOTAL_RECORDS_AUDITED, " Total unsaved:", TOTAL_UNSAVED_AUDITS," Last Saved:", LAST_SAVED )
        except:
            print ("")
    else:
        print ("No results, change search setting")
    
    
  
output = widgets.interactive_output(output_function, {
            'convention_drop_down_value': convention_drop_down,
            'confidence_score_slider_min_value': confidence_score_slider_min,
            'confidence_score_slider_max_value': confidence_score_slider_max,
            'source_drop_down_value': source_drop_down,
            'convention_pos_neg_value' : convention_pos_neg
            })

outputGitHub = widgets.Output()
outputToSave = widgets.Output()
outputCount = widgets.Output()
change_value_button.on_click(change_value_button_clicked)
# keep_value_button.on_click(change_value_button_clicked)
audit_next_button.on_click(audit_next_button_clicked)
save_to_git_button.on_click(save_changes_to_github_button_clicked)
undo_last_audit.on_click(undo_last_audit_button_clicked)
download_button.on_click(download_button_clicked)


In [18]:
#HTML Help Content   
using = '<h4>Using the audit tool</h4><p>This tool will randomly select and display a sentence which has already been classified by the model thus facilitating human evaluation and correction of model predictions. These validated and corrected predictions will be fed back to the model as part of a re-training process, with the objective of increasing model accuracy. The tool will automatically select and display a random sentence from the pool each time an action is involked, either by modifying the filter or by one of the three action buttons [Change], [Ok] and [Skip]. Once the sentence is audited via either the [Ok] or [Change] button it will be used to train the model on the next training iteration.<br/>[Skip]: Use the Skip button if you are unable to confirm the classification (no action will be taken)</p><p>[Ok]: </span> Use the OK button to confirm that the sentence has been classified correctly as [Positive] or [Negative] for the convention in question i.e if the classification is True </p><p>[Chang]e]: Use the change button, if you believe the sentence has been classified incorrectly by the model  i.e if the classification is a "False Positive" or "False Negative".<br/> If the original classification has been "False Positive" and you can correctly classify the sentence as belonging to another convention, please select the appropriate convention via the [Change to] dropdown.</p><p><em>Note: A&nbsp;<strong>GitHub</strong>&nbsp;username and password is required to save the results of the audits, it is important to remember to save the audit on a regular basis</em></p><hr/><p></p>'                              
filtering = '<h4>Filtering</h4>To help obtain a more balanced training data set across all convention models, please review the histogram to determine focus of audit'

#Histogram image
image = Image.open(urllib.request.urlopen(IMAGE_PATH))
image = image_to_byte_array(image)


#keywords tab, populate using keyword_de_dict and keyword_td_dict
td_tab_contents = keyword_td_dict.values() 
de_tab_contents = keyword_de_dict.values() 
tab_name = list(keyword_td_dict.keys())

children = [widgets.HTML(  value="TRAINING DATA: " + str(td_content) + " DOMAIN EXPERT: " +str(de_content)   ) for td_content, de_content in zip (td_tab_contents, de_tab_contents)]

tab_keywords = widgets.Tab()
tab_keywords.children = children
for i in range(len(children)):
    tab_keywords.set_title(i, str(tab_name[i]))

#SET UP A HEADER ACCORDIAN TO SHOW HELP, HISTOGRAM and KEYWORDS
accordion = widgets.Accordion(children=
                              [widgets.HTML(filtering+using),
                               widgets.Image(value=image,format='png',width=1200,height=1200),
                               tab_keywords
                              ]
                         
                             )


accordion.set_title(0, 'About The Tool')
accordion.set_title(1, 'Training Data Histogram')
accordion.set_title(2, 'Convention Keyword hints')
accordion.selected_index = None

In [19]:


display (
        selectHTMLHead,
        accordion,
        space,
        selectHTML,
        widgets.VBox([
        convention_drop_down, 
        source_drop_down, 
        convention_pos_neg,
        confidence_score_slider_min, 
        confidence_score_slider_max]),
        output,
        check_boxes,
        space,
        actionHTML,
        widgets.HBox([change_value_button, audit_next_button, small_space, undo_last_audit]),
        outputCount,
        space,
        saveHTML,
        space,
        widgets.HBox([user, pswd]),
        save_to_git_button,
        space,
        outputGitHub,
        outputToSave,
        space
    )

HTML(value="<font color = '#8c8c8c'><h3><center>*** Model Classification, Audit Tool ***</center></h3></font>"…

Accordion(children=(HTML(value='<h4>Filtering</h4>To help obtain a more balanced training data set across all …

Label(value='  ', layout=Layout(width='100%'))

HTML(value="<font color = '#8c8c8c'><h4><left>--- Filter ---</left></h4></font>")

Output()

...

Output()

Output()

Label(value='  ', layout=Layout(width='100%'))

/Users/aideenf/.pyenv/versions/3.6.0/envs/python3.6/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


ValueError: Plan shapes are not aligned